In [1]:
import cv2
import mediapipe as mp
from deepface import DeepFace

mp_face = mp.solutions.face_detection
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
with mp_face.FaceDetection(min_detection_confidence=0.5) as face_detection:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(rgb)
        if results.detections:
            for det in results.detections:
                box = det.location_data.relative_bounding_box
                h, w, _ = frame.shape
                x, y, w_box, h_box = int(box.xmin*w), int(box.ymin*h), int(box.width*w), int(box.height*h)
                face = frame[y:y+h_box, x:x+w_box]
                if face.size > 0:
                    res = DeepFace.analyze(face, actions=['emotion'], enforce_detection=False)
                    emotion = res[0]['dominant_emotion']
                    cv2.rectangle(frame, (x, y), (x+w_box, y+h_box), (0,255,0), 2)
                    cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
        cv2.imshow("Mediapipe + DeepFace Emotion Detection", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()


ValueError: You have tensorflow 2.19.1 and this requires tf-keras package. Please run `pip install tf-keras` or downgrade your tensorflow.